# OS11 - Exercises 0-6

In [10]:
from sympy import *

In [11]:
import numpy as np

In [12]:
import scipy.linalg as linalg

In [23]:
from itertools import *

## Exercise 0

Build a function that transforms a Node-Node matrix to a Node-Arc matrix.

### Data Introduction

Data will be introduced on a list of (origin, dest, cost) tuples (or any indexable).

In [74]:
datos_originales = []

In [75]:
datos_originales.append((1, 2, 1))
#datos_originales.append((2, 1, 1))   #creo un ciclo a proposito #no es una buena idea
datos_originales.append((1, 3, 1))
datos_originales.append((2, 3, 1))
datos_originales.append((3, 4, 1))
datos_originales.append((4, 1, 1))


In [76]:
datos_originales

[(1, 2, 1), (1, 3, 1), (2, 3, 1), (3, 4, 1), (4, 1, 1)]

### Node-Node Matrix creation

We need to pass from a sparse to a dense matrix formulation.

In [77]:
#Max nodes:
nodes = 0
for d in datos_originales:
    if d[0]> nodes:
        nodes =d[0]
    if d[1] > nodes:
        nodes = d[1]

In [78]:
nodes

4

In [79]:
NN = np.zeros((4,4))

In [80]:
NN

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [81]:
for d in datos_originales:
    NN[d[0]-1, d[1]-1] = d[2]

In [82]:
NN

array([[0., 1., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

### Arc Detection

We create a funtion that returns a list of arcs and weights from NN matrix. Basically reverts to original data.

In [83]:
def arc_detection(NN):
    arcs = []
    #each arc will be represented as a tuple
    for a in ((x,y) for x in range(NN.shape[0]) for y in range(NN.shape[1])):
        if NN[a]>0:
            arcs.append((a, NN[a]))
    return arcs

In [84]:
arcs = arc_detection(NN)

In [85]:
arcs

[((0, 1), 1.0), ((0, 2), 1.0), ((1, 2), 1.0), ((2, 3), 1.0), ((3, 0), 1.0)]

### Arc-Nodes Matrix Creation

Finally we created ANMatrix funtion that receives a n N x N matrix and returns a Nodes x Arcs Matrix.

In [92]:
def ANMatrix(NN):
    arcs = arc_detection(NN)
    MN = np.zeros((NN.shape[0], len(arcs))).astype(int)
    for a in range(len(arcs)):
        MN[arcs[a][0][0], a] = arcs[a][1]
        MN[arcs[a][0][1], a] = -arcs[a][1]
    arcss = [a[0] for a in arcs]
    return MN, arcss

In [93]:
ANMatrix(NN)

(array([[ 1,  1,  0,  0, -1],
        [-1,  0,  1,  0,  0],
        [ 0, -1, -1,  1,  0],
        [ 0,  0,  0, -1,  1]]),
 [(0, 1), (0, 2), (1, 2), (2, 3), (3, 0)])